In [1]:
import numpy as np
from mindspore import Tensor, ops
import mindspore as ms
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from DQAS_tool import Mindspore_ansatz3,vag_nnp3,sampling_from_structure,nmf_gradient,zeroslike_grad_nnp,generate_pauli_string
import numpy as np
import json
import tensorcircuit as tc
import tensorflow as tf
from DQAS_tool import best_from_structure,DQAS_accuracy
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRGenerator
from mindquantum.core.circuit import change_param_name,UN
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
# #Operator Pool


/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [2]:
Pauli_ops=\
['X0 X1','X1 X2','X3 X4','X4 X5','X6 X7','X7 X0',
 'Y0 Y1','Y1 Y2','Y3 Y4','Y4 Y5','Y6 Y7','Y7 Y0',
 'Z0 Z1','Z1 Z2','Z3 Z4','Z4 Z5','Z6 Z7','Z7 Z0',\
 'X0 Y1 Z2 X3','Y1 Z2 X3 X4']

unparameterized_circuit = \
[UN(Z, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(Z, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 UN(X, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(X, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 UN(Y, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(Y, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 ]
ansatz_pr = PRGenerator('ansatz')
parameterized_circuit=[TimeEvolution(QubitOperator(i,ansatz_pr.new()),time=0.5).circuit for i in Pauli_ops]
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=12
shape_nnp = (num_layer,shape_parametized)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [3]:
def best_from_structure(structures: np.array)->Tensor:
    prob = ops.Softmax()(ms.Tensor(structures))
    return ops.Argmax(axis=-1)(prob)

def best_from_structure(structures: np.array)->Tensor:
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [4]:

#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.1))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 50
batch_size=256
avcost1 = 0
ops_onehot = ops.OneHot(axis=-1)
batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率

 
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp3(Structure_params=i,
                                  Ansatz_params=nnp,
                                  paramerterized_pool=parameterized_circuit,unparamerterized_pool=unparameterized_circuit,
                                  num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)

      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    # print(f'loss={infd}\ngrad_nnp={gnnp}\ngrandient_stp={gstp_averge}')
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    cand_preset = best_from_structure(stp)
    best_candidates_history.append(cand_preset.asnumpy())
    

    if epoch % 1 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
        test_ansatz = Mindspore_ansatz3(Structure_p=stp_for_test,
                                        parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,
                                        num_layer=num_layer,
                                        n_qbits=8)
        
        ansatz_parameters=[]
        for layerIndex,i in enumerate(cand_preset):
            if i >=len(parameterized_circuit):
                continue
            else:
                ansatz_parameters.append(nnp[layerIndex,i])
        
        acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
        acc_history.append(acc)
        print(f'二分类准确率 Acc ={acc*100}% ')
        
        

----------epoch 0-----------
batched平均损失:  0.6543136
最好的候选结构: [11 23  5 19  3 15 14 10 24 11 22  4]
二分类准确率 Acc =64.17445482866043% 
----------epoch 1-----------
batched平均损失:  0.6534667
最好的候选结构: [25 23  5  8  3 15 14 10 24 11 22 23]
二分类准确率 Acc =64.58982346832815% 
----------epoch 2-----------
batched平均损失:  0.6533463
最好的候选结构: [25 23  5 14  5 15 14 10 24  4 22 23]
二分类准确率 Acc =64.17445482866043% 
----------epoch 3-----------
batched平均损失:  0.65582854
最好的候选结构: [25 23  5 14  5 15 14 10  0  4 13 23]
二分类准确率 Acc =64.17445482866043% 
----------epoch 4-----------
batched平均损失:  0.6504194
最好的候选结构: [25 23  5 14  5 15 14 10  0  4 13 23]
二分类准确率 Acc =64.17445482866043% 
----------epoch 5-----------
batched平均损失:  0.6526827
最好的候选结构: [25 21  8 14  5  7 14 10  0 11 13 23]
二分类准确率 Acc =65.10903426791276% 
----------epoch 6-----------
batched平均损失:  0.6469898
最好的候选结构: [25 21  8 14  5  7  5 10  0 11 18  2]
二分类准确率 Acc =66.66666666666666% 
----------epoch 7-----------
batched平均损失:  0.64494103
最好的候选结构: [25 21  8 14

In [7]:
structure_distribution_history[-1]

array([[-3.17760695, -1.89972827, -1.27134663, -2.83258734, -0.35378614,
         4.18814072, -2.60725566, -1.99444165, -1.51320048, -1.55333361,
        -0.68687203,  0.38891093, -1.71106915, -0.68653501, -1.21834353,
        -2.17541223, -0.94187029, -0.09086947, -2.81082995, -0.73656952,
        -3.57193257, -1.34947882, -2.50586083,  0.13229891, -3.60354208,
        -0.07253203],
       [ 0.40844915,  0.59206749,  0.68323487,  0.58222385, -0.54116583,
         1.36247507, -1.72570766,  0.70830027, -0.84767879, -0.64351788,
         0.60192883, -0.42996865,  1.35014244, -0.95767841, -0.73856015,
         0.32048661,  1.22358609,  0.40423002, -0.63586893,  0.81879309,
        -0.57868509,  0.95717076, -0.9666075 ,  0.82120449, -1.18295046,
         1.35780089],
       [ 0.45400286, -0.78858699, -1.38317738, -1.23316356, -0.89975035,
         1.00106321,  0.54570557,  0.84520363,  0.92947627, -1.51153631,
        -0.74050868,  1.63690498,  0.72672263,  0.57446498,  0.22012003,
       